# Fitting conductances of whole-cell models

This tutorial


## General approach


- [Models are written in Myokit's MMT syntax](https://myokit.readthedocs.io/syntax/index.html), usually by [downloading a CellML model](https://models.cellml.org/electrophysiology) and then [importing it](https://myokit.readthedocs.io/api_formats/cellml.html).
- [Simulations are run](https://myokit.readthedocs.io/api_simulations/Simulation.html) using the `Simulation` class, which uses CVODE to solve the ODEs.
  - For ion-channel stuff with piecewise constant voltage-step protocols (no ramps or sine waves) it's usually faster to use Myokit's [Hodgkin-Huxley model](https://myokit.readthedocs.io/api_library/hh.html) or [Markov model](https://myokit.readthedocs.io/api_library/markov.html) classes to run analytical simulations)
- [A pints.ForwardModel](https://nbviewer.jupyter.org/github/pints-team/pints/blob/master/examples/writing-a-model.ipynb) is wrapped around a Myokit simulation.
- [A pints.ErrorMeasure](https://pints.readthedocs.io/en/latest/error_measures.html) or [pints.LogLikelihood](https://pints.readthedocs.io/en/latest/log_likelihoods.html) is defined
- [Optimisation](https://nbviewer.jupyter.org/github/pints-team/pints/blob/master/examples/optimisation-first-example.ipynb) or [Bayesian inference](https://nbviewer.jupyter.org/github/pints-team/pints/blob/master/examples/sampling-first-example.ipynb) is run.



## Fitting to action potential

## Fitting to action potential and calcium transient

In [1]:
import numpy as np
import pints
import myokit
import matplotlib.pyplot as plt

class MyokitModel(pints.ForwardModel):
    """
    This is a pints model, i.e. a statistical model that takes parameters and
    times as input, and returns simulated values.
    """
    def __init__(self):
        m, p, _ = myokit.load('resources/beeler-1977.mmt')
        self.simulation = myokit.Simulation(m, p)

    def n_parameters(self):
        return 5

    def n_outputs(self):
        return 2

    def simulate(self, parameters, times):

        self.simulation.reset()
        self.simulation.set_constant('ina.gNaBar', parameters[0])
        self.simulation.set_constant('ina.gNaC', parameters[1])
        self.simulation.set_constant('isi.gsBar', parameters[2])
        self.simulation.set_constant('ik1.gK1', parameters[3])
        self.simulation.set_constant('ix1.gx1', parameters[4])

        log = self.simulation.run(
            times[-1] + 1,
            log_times = times,
            log = ['engine.time', 'membrane.V', 'calcium.Cai'],
        )
        return np.vstack((log['membrane.V'], log['calcium.Cai'])).T


# Create a model
model = MyokitModel()

# Generate some 'experimental' data
x_true = np.array([4, 0.003, 0.09, 0.35, 0.8])
times = np.linspace(0, 600, 601)
values = model.simulate(x_true, times)
print(values.shape)

# Add noise
noisy_values = np.array(values, copy=True)
noisy_values[:, 0] += np.random.normal(0, 1, values[:, 0].shape)
noisy_values[:, 1] += np.random.normal(0, 5e-7, values[:, 1].shape)

plt.figure()
plt.suptitle('Generated data')
plt.subplot(2, 1, 1)
plt.xlabel('Time (ms)')
plt.ylabel('Vm (mV)')
plt.plot(times, noisy_values[:, 0])
plt.plot(times, values[:, 0])
plt.subplot(2, 1, 2)
plt.xlabel('Time (ms)')
plt.ylabel('[Ca]i (mol/L)')
plt.plot(times, noisy_values[:, 1])
plt.plot(times, values[:, 1])
#plt.show()

# Create an object with links to the model and time series
problem = pints.MultiOutputProblem(model, times, noisy_values)

# Create a score function
weights = [1 / 70, 1 / 0.000006]
score = pints.SumOfSquaresError(problem, weights=weights)

# Select some boundaries
lower = x_true / 2
upper = x_true * 2
boundaries = pints.RectangularBoundaries(lower, upper)

# Perform an optimization
x0 = x_true * 1.2
optimiser = pints.OptimisationController(
    score, x0, boundaries=boundaries, method=pints.CMAES)

print('Running...')
x_found, score_found = optimiser.run()

# Compare parameters with original
print('Found solution:          True parameters:' )
for k, x in enumerate(x_found):
    print(pints.strfloat(x) + '    ' + pints.strfloat(x_true[k]))

fitted_values = problem.evaluate(x_found)

plt.figure()
plt.suptitle('Generated data + simulation with fit parameters')
plt.subplot(2, 1, 1)
plt.xlabel('Time (ms)')
plt.ylabel('Vm (mV)')
plt.plot(times, noisy_values[:, 0])
plt.plot(times, fitted_values[:, 0])
plt.subplot(2, 1, 2)
plt.xlabel('Time (ms)')
plt.ylabel('[Ca]i (mol/L)')
plt.plot(times, noisy_values[:, 1])
plt.plot(times, fitted_values[:, 1])
plt.show()


(601, 2)
Running...
Minimising error measure
Using Covariance Matrix Adaptation Evolution Strategy (CMA-ES)
Running in sequential mode.
Population size: 8
Iter. Eval. Best      Time m:s
0     8      59.67956   0:00.1
1     16     18.16472   0:00.2
2     24     14.10243   0:00.3
3     32     14.10243   0:00.3
20    168    9.152392   0:01.1
40    328    9.063908   0:01.8
60    488    9.061171   0:02.5
80    648    9.053362   0:03.4
100   808    9.053362   0:04.1
120   968    8.969122   0:04.9
140   1128   8.879653   0:05.6
160   1288   8.807195   0:06.4
180   1448   8.524149   0:07.2
200   1608   7.718861   0:08.0
220   1768   7.680624   0:08.8
240   1928   7.677583   0:09.6
260   2088   7.677583   0:10.4
280   2248   7.677583   0:11.1
300   2408   7.677583   0:11.8
320   2568   7.677583   0:12.6
340   2728   7.677583   0:13.4
360   2888   7.677583   0:14.2
380   3048   7.677583   0:15.0
400   3208   7.677583   0:15.7
420   3368   7.677576   0:16.4
440   3528   7.677558   0:17.2
460   36

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

## Summary